## Setup environment

In [2]:
# mount driver
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Seg

# install dependents
!pip install -q "monai-weekly[nibabel, tqdm, einops]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

# install tensorboard-plugin-3d
!pip install -q tensorboard-plugin-3d

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Seg
     |████████████████████████████████| 835 kB 4.2 MB/s 
     |████████████████████████████████| 27.1 MB 81.3 MB/s 
     |████████████████████████████████| 462 kB 60.8 MB/s 


In [3]:
import os
from pathlib import PurePath

from monai.utils import first
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AsDiscrete,
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    ToTensord,
    LabelFilterd,
    MapLabelValued
)
from monai.data import (
    DataLoader,
    CacheDataset,
    decollate_batch
)
from monai.visualize import plot_2d_or_3d_image

import torch
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Setup config

In [4]:
class Config:
  model_name = 'visualization'
  use_pretrained_model = True
  root_dir = './'
  data_dir = ''
  model_dir = os.path.join(root_dir, 'models', model_name)
  log_dir = os.path.join(root_dir, 'logs', model_name)
  eval_dir = os.path.join(root_dir, 'evals', model_name)
  pretrained_model_pth = os.path.join(model_dir, 'dc_88_best_metric_model.pth')
  batch_size = 1
  num_workers = 0
  cache_rate = 1.0
  pin_memory = False


class LoaclConfig(Config):
  data_dir = 'D:\\home\\school\\ntut\\dataset\\SegTHOR\\train'

class ColabConfig(Config):
  root_dir = './'
  data_dir = os.path.join(root_dir, 'dataset/SegTHOR/train')
  num_workers = 2
  pin_memory = True

is_colab = True

if is_colab:
  config = ColabConfig
else:
  config = LoaclConfig

# setup dir
root_dir = config.root_dir
data_dir = config.data_dir
model_dir = config.model_dir
log_dir = config.log_dir
eval_dir = config.eval_dir

# setup data loader param
batch_size = config.batch_size
num_workers = config.num_workers
cache_rate = config.cache_rate
pin_memory = config.pin_memory

# show dir
print(f'root_dir: {root_dir}')
print(f'data_dir: {data_dir}')
print(f'model_dir: {model_dir}')
print(f'log_dir: {log_dir}')
print(f'eval_dir: {eval_dir}')

root_dir: ./
data_dir: ./dataset/SegTHOR/train
model_dir: ./models/visualization
log_dir: ./logs/visualization
eval_dir: ./evals/visualization


## Setup transforms

In [ ]:
eval_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        LabelFilterd(keys=["label"], applied_labels=[2]),
        MapLabelValued(keys=["label"], orig_labels=[0,2], target_labels=[0,1]),
        AddChanneld(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(
            keys=["image"], a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True
        ),
        CropForegroundd(keys=['image','label'], source_key='label',margin=40),
        ToTensord(keys=["image", "label"]),
    ]
)

## Setup dataset and data loader

In [6]:
def get_ds_pths(data_dir):
    img_pths = []
    lbl_pths = []
    for patient_dir in sorted(os.listdir(data_dir)):
        img_pths.append(os.path.join(data_dir, patient_dir, f'{patient_dir}.nii.gz'))
        lbl_pths.append(os.path.join(data_dir, patient_dir, f'GT.nii.gz'))
    return img_pths, lbl_pths


def get_data_dicts(data_dir):
    img_pths, lbl_pths = get_ds_pths(data_dir)
    data_dicts = [
        {"image": img_pth, "label": lbl_pth}
        for img_pth, lbl_pth in zip(img_pths, lbl_pths)
    ]
    return data_dicts

In [7]:
data_dicts = get_data_dicts(data_dir)
train_files, val_files = data_dicts[:10], data_dicts[10:15]

In [8]:
eval_tr_ds = CacheDataset(
    data=train_files,
    transform=eval_transforms,
    cache_rate=cache_rate,
    num_workers=num_workers
)

eval_tr_loader = DataLoader(
    eval_tr_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=pin_memory
)

eval_val_ds = CacheDataset(
    data=val_files,
    transform=eval_transforms,
    cache_rate=cache_rate,
    num_workers=num_workers
)

eval_val_loader = DataLoader(
    eval_val_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=pin_memory
)

Loading dataset: 100%|██████████| 5/5 [00:36<00:00,  7.30s/it]


## Plot in with matplot as well as TensorBoard

In [ ]:
def plot(data_loader, log_dir, type):
  for i, val_data in enumerate(data_loader):
    # Plot the data for Tensorboard as well
    print(f'shapes: {val_data["image"].shape} {val_data["label"].shape}')
    # Visualize the input image and label
    sw = SummaryWriter(log_dir=log_dir)
    plot_2d_or_3d_image(data=val_data["image"], step=0, writer=sw, frame_dim=-1, tag=f"{type}_image")
    plot_2d_or_3d_image(data=val_data["label"], step=0, writer=sw, frame_dim=-1, tag=f"{type}_label")
    break

In [ ]:
plot(eval_tr_loader, log_dir, type="tr")

shapes: torch.Size([1, 1, 172, 166, 120]) torch.Size([1, 1, 172, 166, 120])


In [ ]:
%tensorflow_version 2.x

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=logs/visualization